In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
from mdp import MDPEnv, is_valid
from tqdm.contrib.concurrent import process_map
from tqdm import tqdm
import pickle
import gzip
from functools import cache
import numpy as np
from constants import MAP_SIZE, HAZARD_P, SLIP_P, MYOPIC, SEEDS

In [11]:
def curr(myo, teach):
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
    model = tf.keras.models.load_model(f'/storage1/fs1/chien-ju.ho/Active/gym/models/myopic_{myo}.keras')
    
    
    data = np.load(f'/storage1/fs1/chien-ju.ho/Active/gym/data/test/myopic_{teach}.npz')
    x, y = data['x'], data['y']    
    bs=1024
    for b in range(0, bs*10, bs):
        model.fit(x[b:b+bs], y[b:b+bs], verbose=0)
        model.save(f'/storage1/fs1/chien-ju.ho/Active/gym/models/curr/myopic_1_{teach}_{b+bs}.keras')

In [12]:
%%time
myos = [1 for _ in MYOPIC]
teachs = [teach for teach in MYOPIC]
process_map(curr, myos, teachs)

  0%|          | 0/7 [00:00<?, ?it/s]

CPU times: user 31.6 ms, sys: 222 ms, total: 253 ms
Wall time: 8.29 s


[None, None, None, None, None, None, None]

In [33]:
def model_reward(model, state):
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
    tf.get_logger().setLevel('ERROR')
    
    env = MDPEnv(map_size=MAP_SIZE, hazard_p=0.3, slip_p=0.4)
    env.reset(state)
    
    agents = []
    grids = []
    target, hazards, rand = env.state['target'], env.state['hazards'], env.state['rand']
    for i in range(MAP_SIZE):
        for j in range(MAP_SIZE):
            if (i,j) == target or (i,j) in hazards:
                continue
            state = ((i,j), target, hazards, rand)
            env.reset(state)
            agents.append((i,j))
            grids.append(env.get_grid())
            
    model = tf.keras.models.load_model(f'/storage1/fs1/chien-ju.ho/Active/gym/models/curr/myopic_1_{model}.keras')
    actions = model.predict(np.array(grids)).argmax(axis=1)
    trans = dict(zip(agents, actions))
    
    rewards = []
    for i in range(1000):
        env.reset(state)
        reward_sum = 0
        for step in range(100):
            action = trans[env.state['agent']]
            _, reward, terminated, truncated, _ = env.step(action)
            reward_sum += reward
            if terminated or truncated:
                break
        rewards.append(reward_sum)
    return np.mean(rewards)

In [34]:
data = np.load(f'/storage1/fs1/chien-ju.ho/Active/gym/data/test/myopic_{1}.npz')
x, y = data['x'], data['y']
x_states = x[-20000:]

In [35]:
all_res = {}

In [ ]:
%%time
for teach in MYOPIC:
    models = [f'{teach}_1024' for _ in x_states]
    res = process_map(model_reward, models, x_states, chunksize=1)
    print(teach, np.mean(res), np.std(res))
    all_res[models[0]] = res

  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
%%time
for teach in MYOPIC:
    models = [f'curr/myopic_1_{teach}_10240' for _ in x_states]
    res = process_map(model_reward, models, x_states, chunksize=1)
    print(teach, np.mean(res), np.std(res))
    all_res[models[0]] = res

In [ ]:
for k,v in all_res.items():
    print(k, len(v))